In [1]:
!pip install opencv-python scikit-learn pyyaml ultralytics tqdm

In [ ]:
import os
import cv2
import shutil
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Define dataset paths
base_dir = 'C:/Users/Gaurav Lodhi/Downloads/Winners/datasetss'
train_dir = 'dataset/images/train/'
val_dir = 'dataset/images/val/'
test_dir = 'dataset/images/test/'
train_labels_dir = 'dataset/labels/train/'
val_labels_dir = 'dataset/labels/val/'
test_labels_dir = 'dataset/labels/test/'

# Create directories for images and labels
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)
os.makedirs(test_labels_dir, exist_ok=True)

# Splitting ratio
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Class names
class_names = [
    'Vegetable_Fresh_zucchini', 'Vegetable_Fresh_Okra', 'Vegetable_Fresh_Cuccumber',
    'Vegetable_Fresh_Chilli', 'stale_tomato', 'stale_watermelon', 'Stale_Strawberry',
    'Stale_Spanich', 'stale_potato', 'stale_pomogrante', 'stale_orange', 'stale_onion',
    'stale_okra', 'stale_mango', 'Stale_Lulo', 'stale_lemon', 'stale_chilli',
    'stale_capsicum', 'stale_beetroot', 'stale_bitter_ground', 'stale_avacado',
    'stalecucumber', 'Stalepear', 'staleapple', 'Fruit_Fresh_Banana', 'pearfresh',
    'Fresh_Watermelon', 'Fruit_Fresh_Avacado', 'fresh_tomato', 'Fresh_Tamarillo',
    'Fresh_Strawberry', 'fresh_Spanich', 'fresh_potato', 'fresh_pomegrante',
    'fresh_orange', 'fresh_onion', 'fresh_lulo', 'fresh_lemon', 'fresh_eggplant',
    'fresh_cauliflower', 'fresh_capsicum', 'fresh_carrot', 'fresh_bitter_gourd',
    'Fresh_cabbage', 'fresh_beetroot', 'freshapple'
]

# Map each class name to a numeric label
class_map = {name: idx for idx, name in enumerate(class_names)}

# Count total number of images for the progress bar
total_images = 0
folders = [folder for folder in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, folder))]

for folder in folders:
    folder_path = os.path.join(base_dir, folder)
    total_images += len([img for img in os.listdir(folder_path) if img.endswith(('.jpg', '.jpeg', '.png'))])

# Function to annotate images and move them to the correct folder
def annotate_and_move_images(images, folder_path, dest_dir, labels_dir, class_id, folder_name_index):
    for img in images:
        image_path = os.path.join(folder_path, img)
        
        # Rename the image to include the folder index
        dest_image_name = f"{folder_name_index}_{img}"  # Include folder index in the image name
        dest_image_path = os.path.join(dest_dir, dest_image_name)
        shutil.copy(image_path, dest_image_path)

        # Load the image to get dimensions
        img_data = cv2.imread(image_path)
        if img_data is None:
            print(f"Warning: Couldn't read image {img}")
            continue

        height, width, _ = img_data.shape

        # Create YOLO format annotation (whole image)
        x_center = 0.5
        y_center = 0.5
        obj_width = 1.0
        obj_height = 1.0
        annotation = f"{class_id} {x_center} {y_center} {obj_width} {obj_height}\n"

        # Save the annotation with the updated name
        annotation_name = dest_image_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt')
        annotation_path = os.path.join(labels_dir, annotation_name)
        with open(annotation_path, 'w') as f:
            f.write(annotation)

# Iterate over each folder (class) with a progress bar for overall progress
with tqdm(total=total_images, desc="Copying and Annotating all images") as pbar:
    for folder_index, folder in enumerate(folders):
        folder_path = os.path.join(base_dir, folder)

        images = [img for img in os.listdir(folder_path) if img.endswith(('.jpg', '.jpeg', '.png'))]
        if len(images) == 0:
            continue

        train_imgs, temp_imgs = train_test_split(images, test_size=(1 - train_ratio), random_state=42)
        val_imgs, test_imgs = train_test_split(temp_imgs, test_size=test_ratio / (test_ratio + val_ratio), random_state=42)

        # Get the class ID from the folder name using the class map
        class_id = class_map.get(folder)
        if class_id is None:
            print(f"Error: Class not found for folder {folder}")
            continue

        # Annotate and move images for train, val, and test sets
        annotate_and_move_images(train_imgs, folder_path, train_dir, train_labels_dir, class_id, folder_index)
        annotate_and_move_images(val_imgs, folder_path, val_dir, val_labels_dir, class_id, folder_index)
        annotate_and_move_images(test_imgs, folder_path, test_dir, test_labels_dir, class_id, folder_index)

        pbar.update(len(images))

print("Data split and annotation process completed.")

Copying and Annotating all images:  10%|█         | 4452/44287 [00:46<16:43, 39.71it/s] 

In [1]:
from ultralytics import YOLO
import os

# Define dataset config for YOLOv8
data_yaml = """
train: C:/Users/Gaurav Lodhi/Downloads/Winners/GRID_test1/dataset/images/train
val: C:/Users/Gaurav Lodhi/Downloads/Winners/GRID_test1/dataset/images/val
test: C:/Users/Gaurav Lodhi/Downloads/Winners/GRID_test1/dataset/images/test

nc: 46  # Number of classes
names: [
    'Vegetable_Fresh_zucchini',
    'Vegetable_Fresh_Okra',
    'Vegetable_Fresh_Cuccumber',
    'Vegetable_Fresh_Chilli',
    'stale_tomato',
    'stale_watermelon',
    'Stale_Strawberry',
    'Stale_Spanich',
    'stale_potato',
    'stale_pomogrante',
    'stale_orange',
    'stale_onion',
    'stale_okra',
    'stale_mango',
    'Stale_Lulo',
    'stale_lemon',
    'stale_chilli',
    'stale_capsicum',
    'stale_beetroot',
    'stale_bitter_ground',
    'stale_avacado',
    'stalecucumber',
    'Stalepear',
    'staleapple',
    'Fruit_Fresh_Banana',
    'pearfresh',
    'Fresh_Watermelon',
    'Fruit_Fresh_Avacado',
    'fresh_tomato',
    'Fresh_Tamarillo',
    'Fresh_Strawberry',
    'fresh_Spanich',
    'fresh_potato',
    'fresh_pomegrante',
    'fresh_orange',
    'fresh_onion',
    'fresh_lulo',
    'fresh_lemon',
    'fresh_eggplant',
    'fresh_cauliflower',
    'fresh_capsicum',
    'fresh_carrot',
    'fresh_bitter_gourd',
    'Fresh_cabbage',
    'fresh_beetroot',
    'freshapple'
]
"""

# Save config to data.yaml file
data_yaml_path = 'C:/Users/Gaurav Lodhi/Downloads/Winners/GRID_test1/dataset/data.yaml'
os.makedirs(os.path.dirname(data_yaml_path), exist_ok=True)

with open(data_yaml_path, 'w') as f:
    f.write(data_yaml)

# Load YOLOv8 model and train
model = YOLO('yolov8n.pt')  # Load the YOLOv8 nano model

# Train the model
model.train(data=data_yaml_path, epochs=10, imgsz=640, batch=32)
print("Training completed.")

# Validate the model
results = model.val(data=data_yaml_path, imgsz=640)

# Print validation results
print("Validation results:", results)

New https://pypi.org/project/ultralytics/8.3.13 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.12  Python-3.12.4 torch-2.4.1+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/Users/Gaurav Lodhi/Downloads/Winners/GRID_test1/dataset/data.yaml, epochs=10, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=Non

train: Scanning C:\Users\Gaurav Lodhi\Downloads\Winners\GRID_test1\dataset\labels\train.cache... 58175 images, 5978 backgrounds, 3875 corrupt: 100%|██████████| 64153/64153 [00:00<?, ?it/s]

train: WARNING  C:\Users\Gaurav Lodhi\Downloads\Winners\GRID_test1\dataset\images\train\0_Screen Shot 2018-06-08 at 4.59.36 PM.png: ignoring corrupt image/label: Label class 45 exceeds dataset class count 40. Possible class labels are 0-39
train: WARNING  C:\Users\Gaurav Lodhi\Downloads\Winners\GRID_test1\dataset\images\train\0_Screen Shot 2018-06-08 at 4.59.49 PM.png: ignoring corrupt image/label: Label class 45 exceeds dataset class count 40. Possible class labels are 0-39
train: WARNING  C:\Users\Gaurav Lodhi\Downloads\Winners\GRID_test1\dataset\images\train\0_Screen Shot 2018-06-08 at 4.59.57 PM.png: ignoring corrupt image/label: Label class 45 exceeds dataset class count 40. Possible class labels are 0-39
train: WARNING  C:\Users\Gaurav Lodhi\Downloads\Winners\GRID_test1\dataset\images\train\0_Screen Shot 2018-06-08 at 5.00.03 PM.png: ignoring corrupt image/label: Label class 45 exceeds dataset class count 40. Possible class labels are 0-39
train: WARNING  C:\Users\Gaurav Lodhi\Do


val: Scanning C:\Users\Gaurav Lodhi\Downloads\Winners\GRID_test1\dataset\labels\val.cache... 16998 images, 1694 backgrounds, 1356 corrupt: 100%|██████████| 18692/18692 [00:00<?, ?it/s]

val: WARNING  C:\Users\Gaurav Lodhi\Downloads\Winners\GRID_test1\dataset\images\val\0_Screen Shot 2018-06-08 at 5.00.03 PM.png: ignoring corrupt image/label: Label class 45 exceeds dataset class count 40. Possible class labels are 0-39
val: WARNING  C:\Users\Gaurav Lodhi\Downloads\Winners\GRID_test1\dataset\images\val\0_Screen Shot 2018-06-08 at 5.00.12 PM.png: ignoring corrupt image/label: Label class 45 exceeds dataset class count 40. Possible class labels are 0-39
val: WARNING  C:\Users\Gaurav Lodhi\Downloads\Winners\GRID_test1\dataset\images\val\0_Screen Shot 2018-06-08 at 5.00.26 PM.png: ignoring corrupt image/label: Label class 45 exceeds dataset class count 40. Possible class labels are 0-39
val: WARNING  C:\Users\Gaurav Lodhi\Downloads\Winners\GRID_test1\dataset\images\val\0_Screen Shot 2018-06-08 at 5.00.50 PM.png: ignoring corrupt image/label: Label class 45 exceeds dataset class count 40. Possible class labels are 0-39
val: WARNING  C:\Users\Gaurav Lodhi\Downloads\Winners\GR

Plotting labels to runs\detect\train12\labels.jpg... 


: 